计算机视觉三大任务
- 目标检测
- 目标分割
- 目标跟踪


# 目标跟踪 Object Tracking

# 历史发展

目标跟踪是一个很大的领域

在deeplearning没有出现的时候就有跟踪了

2010-2012 TLD track learning detection

2012 corlatin filter相关滤波

2012之后 Siamese基于孪生, 基于CNN

甚至认为跟踪就是检测的复合任务

# 跟踪简述
## 什么是跟踪
- 很多时候应该叫做 visual tracking
    - 比如颜色, 没有物体出现的区域
- 还有 target tracking 
    - 比如敌方飞机, 这是在object出现之前就已经知道它是什么了
- 甚至还有 把tracking换成tracing(轨迹)
- 第一帧如果没有任何物体, 从某一时刻开始出现我们感兴趣的物体, 这时候也需要recognition


强调在很多物体的表现很好, 而不是固定种类

比如ufo从来没见过, oneshot learning

难点: 视角, 姿态,运动,  重叠, 遮挡, 多个相似, 抖动, 亮度, 变色, 突然消失, 要鲁棒性好

实时性(评价指标: mIOU)

## 简介视频

re3: Real-Time Recurrent Regression Networks for Visual Tracking of Generic Objects

[re3](https://gitlab.cs.washington.edu/xkcd/re3-tensorflow)


## 输入输出

输入是图片序列sequence, 是有顺序的; 而不是集合set, 不是一堆图片

通常, 输入还应该有第一帧目标的位置

具体输入形式,有9种

比如中心点, 轮廓上12个点, bbox, 斜的bbox, 全部轮廓, 内部填充物mask, 骨架, 等等

输出接下来每一帧目标可能的位置

## 应用场景

- 工业机器人
- 监控
- 交通
- VR(包括传感器)
- 需要其他信号,比如gps, 声纳

## 分类&分支方向

按目标多少: 多目标, 单目标(这两个领域完全不同)

第一帧是否给定(企业级往往没有)

Model-Free

![1](1.PNG)

- Environment:
    - 如果是真实场景, 就需要我们人工拍摄
    - 约束环境: 虚拟几何体, 利用摄影知识,完成环境变化
- 摄像头Camera
    - 多个视角
    - 其他摄像头,红外,声纳
- 失效性
    - 长时跟踪
    - 短时跟踪(目标消失)
- 物体种类与数量
- 数据集
- 指标

## 挑战
1. Motion blur 运动模糊 ,边界不清楚
2. Occlusion 遮挡(框如何变化)
3. Deformation 变形,如跳舞的姿态
4. Scale variation 尺度, 大小, 车跑过来
5. Move too fast 运动太快可能不连续
6. Exposure change 曝光点变化

最重要的: 缺少训练样本

> 肯定要增广!!

# 分类

多目标

单目标

这两个完全不同的方向, 没办法用一个方案同时解决两个


第一帧是否给定: 一般给定

在企业业务中可能没有


# 主要的挑战
框的大小变化

本身姿态变形(跳舞,弯腰)

尺度变化屏幕占比(车远近,)


最大的困难:缺少训练样本

需要Augmentation




# 问题的简化

## 简化成detection
每一帧都提出来:
- 可能闪烁 可能有些帧没有
- 损失信息.导致输出不准确,不能考虑前后的连贯性

弥补:
- 比如前后帧差别太大, 剔除
- 比如前后帧消失,插补

## 背景固定

背景的像素固定(比如监控录像, 只有光照略微变化)

关键是找到背景如何变化的, 第二帧减去第一帧:

### 线性变换法

- 具体来说, 应该找到两张图片的关键点
- 发现第一帧很多关键点经过单应性矩阵变换到了第二帧
- 此时应该 2 - H * 1

### 帧插法

单纯的第二帧减第一帧

做阈值筛出变化的东西, 就是运动的物体

### 光流法

背景可能存在一种不为人知的变化

把背景的每个像素点当作光点, 每时每刻发生变化

预测光流场在下一个时刻怎么变化

给函数带入t值, 得到背景, 每帧减去背景即可

\(个人想法: 拉格朗日法和欧拉法\)

### 背景削弱



### 帧插法 演示

In [ ]:
import cv2
import os
import numpy as np

In [ ]:
root_path = os.curdir
video_path = os.path.join(root_path, "long.mp4")

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib
# matplotlib.use('Qt5Agg')

In [ ]:
cap = cv2.VideoCapture(video_path)
frame_num = 0
# plt.ion()
while cap.isOpened():
    ret_status, frame = cap.read()
    frame_num+=1
    if ret_status:
        temp_frame = frame
        if frame_num ==1:
            prev_frame = cv2.cvtColor(temp_frame, cv2.COLOR_BGR2GRAY)
        if frame_num>2:
            curr_frame = cv2.cvtColor(temp_frame, cv2.COLOR_BGR2GRAY)
            curr_frame = cv2.absdiff(curr_frame,prev_frame)


            # 阈值变化处理, 二池化操作, 放大
            # 对于40分界, 超过40就绝对亮
            # 这里要把一些背景中变化的东西剔除调
            _, threshold_frame = cv2.threshold(curr_frame,40,255,cv2.THRESH_BINARY)

            # 降噪
            gaussian_frame = cv2.GaussianBlur(threshold_frame, (3,3),0)

            img, contours,hia = cv2.findContours(gaussian_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # print(contours)
            for c in contours:
                if cv2.contourArea(c)>=150:
                    (x,y,w,h) = cv2.boundingRect(c)
                    cv2.rectangle(frame,(x,y),(w+x,y+h),(255,255,0),2)
            # plt.subplot(121)
            # plt.imshow(curr_frame)
            # plt.subplot(122)
            # plt.imshow(threshold_frame)
            # plt.pause(0.033)
            cv2.imshow('ORIGIN',cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            cv2.imshow('ABSDIFF', curr_frame)
            cv2.imshow('BINARY', threshold_frame)
            cv2.imshow('GAUSSIAN', gaussian_frame)

            if cv2.waitKey(33) & 0xFF==ord('q'):
                break
        prev_frame = cv2.cvtColor(temp_frame, cv2.COLOR_BGR2GRAY)
cap.release()   
cv2.destroyAllWindows()

# 跟踪器

跟踪器有上百种, 留下来的也就几种

实时性很关键(绝大多数情况下, 都是在线跟踪, 只知道当前帧和历史帧)

![2](2.PNG)

主要分4类
- Dataset
    - VOT竞赛
    - OTB object tracking benchmark
    - 其他
- Others
    - 静态背景
    - RL
    - ...
- 相关滤波
    - KCF
    - CNN
- CNN
    - 基于检测
    - 孪生网络


## 相关滤波

## 使用了核方法的相关滤波
快

离散傅里叶, 对于原始样本增强和采样的速度大大提高

循环矩阵最小化

用了HOG特征

## 卡尔玛滤波 Kalman filter

> 广泛用在控制和航天

### CCOT

### MD NET

1:56

## Siam R-CNN 2019 基于检测

[论文地址](https://arxiv.org/abs/1911.12836v1)

![3](3.PNG)

直接使用检测的backbone

孪生的意义:

    第一帧, 当前帧都使用了deep feature extractor

    而且经过的时候权值不变(shared weight)

    两个输入进入, 两个输出

1. 第一帧经过backbone生成特征图
2. 把第一帧的bbox信息, 投影到特征图上, 并使用ROI align,得到固定大小的一个特征图
3. 当前帧进入backbone生成特征图, 特征图进入region proposal network, 得到区域建议, 也使用ROI align,得到多个固定大小的特征图
4. 将3的结果分别和2的结果concat, 先re-detection,也就是再回归, 分数越高, 说明越可能是第一帧出现的
5. 对4的结果进行筛查,考虑前几帧, 大小夸张的去掉, 离得远的去掉等等
6. 用5的结果, 使用4中同样的re-detection网络, 再检测, 得到更加真实的框
7. 对6的结果进行一些动态的预估, 并且插值放大, 放到原图上
8. (可选)把7的结果进行分割, 画出物体的边缘

## 相关滤波 Correlation Filters

### 什么是"相关"
相关, 类似于协方差

比如大图中有个小人, 一张小图就是那个小人

拿小图和大图卷积, 得到score map

是响应分数, 也是相关操作

### 真实情况

但是这种相关操作很慢, 用点乘更好

傅里叶变换可以把卷积变成乘法

小图其实是第一帧出现的目标

大图小图分别做傅里叶变换, 然后点乘

点乘结果逆傅里叶变换, 得到结果

相当于在时域频域做了变换

### MOSSE 2010

minimum output sum of square error
 
最小方差输出的滤波器

![4](4.PNG)

Gi是响应图

G = f 卷积 h

f 是大图, h 是小图

目的: 找到一个合适的h, 使得每一帧, 卷积的时候, g在目标区域达到最大, 接近gt

也就使下式最小:
$$ \sum_{i=1}^{N}|f_{i} * h -g_{gt_{i}}|^2$$

将f,g,h傅里叶变换:
$$ \underset{H}{argmin}\sum_{i=1}^{N}|F_{i} \odot H -G_{i}|^2$$

求导可解共轭

$$ H^{*} = \frac{\sum_{i}^{N}G_{i}\odot F_{i}^{*}}{\sum_{i}^{N}F_{i}\odot F_{i}^{*}}$$

线上更新, 不可能拿到所有帧

只能用之前的:

$$ H_{i}^{*} = \eta\frac{G_{i}\odot F_{i}^{*}}{F_{i}\odot F_{i}^{*}}+(1-\eta ) H_{i-1}^{*}$$

实际上还对每一帧作了增广


### 核相关滤波器 Kernelized Correlation Filters

#### 循环矩阵: 

用来增广样本

比如当样本是一维的:

![5](5.PNG)

三维图片:

![6](6.PNG)

#### 循环矩阵的性质:

可以对角化

![7](7.PNG)

#### KCF算法:

第一帧,训练一个滤波器, 使得第一帧在目标位置响应最强

第二帧同样,  用循环矩阵增广, 训练一个滤波器, 使得之前所有找到的更准确

## SiamFC 2015

[论文](https://arxiv.org/abs/1606.09549)

![8](8.PNG)


目标和当前帧同时送入backbone

backbone不训练

两者的输出做相关

得到响应图

## SiamRPN

[论文](https://arxiv.org/abs/1606.09549)

![9](9.PNG)

模板帧和检测帧都进入CNN

然后结果一个分支做分类(是还是不是)

一个分支做回归

## SiamMask 2019竞赛亚军

[视频](https://www.bilibili.com/video/BV1Kt411u7CT)

[论文](http://www.robots.ox.ac.uk/%7Eqwang/SiamMask/)

[开源](https://github.com/foolwood)

![10](10.PNG)

![11](11.PNG)

作者认为: 跟踪精度上限, 不是由框决定, 由分割决定

单纯跟踪框精度不行的

当前帧和目标值也是进入相同的backbone, 得到特征图, 做相关操作

每一个长条都是响应分数

256通道可以做成63x63,做成mask

生成mask的过程类似于unet, 多尺度融合




## SiamRPN, SiamMask, KalmanFilter还在用

## Kalman filter

控制算法

转变思维: 估算方法, 使得我们估算的东西方差最小

适用于: 线性系统+ 高斯噪声 (非高斯 粒子滤波)


场景: 小车在走, 我们在路边不动, 每个时刻都观测它

t时刻, 观测值, 距离zt, 这是个标量, 小车的状态xt=\[position_t, vt\]

zt未必等于position_t, 有误差

zt是均值, R是协方差矩阵(或者标量方差)

根据牛顿第二定律，我们其实可以知道上一时刻

$$p_{t}=p_{t-1}+\Delta t \cdot v_{t} + \frac{1}{2}\Delta t^2 \cdot a_{t}$$

$$v_{t} = v_{t-1} + \Delta t \cdot a_{t}$$

状态转移方程：


$$x_{t}=\begin{bmatrix}1&\Delta t \\0&1\end{bmatrix}x_{t-1}+\begin{bmatrix}\frac{1}{2}\Delta t^2 \\ \Delta t\end{bmatrix}a_{t}$$

$$x_{t}=F_{t}\cdot x_{t-1} + B_{t}\cdot u_{t}$$

F是状态转移矩阵， B是控制矩阵

协方差：
$$P_{t}=F_{t}P_{t-1}F_{t}^{T}$$

以上两个都是预测值，预测不准

应该有：

$$P_{t}=F_{t}P_{t-1}F_{t}^{T}+Q$$

观测方程：

$$z_{t} = H\cdot x_{t} +V$$

V是误差， 没有速度信息所以H=\[1, 0\]

状态更新：

$$\hat{x_{t}}^{kalman}=x_{t}+K_{t}(z_{t}-H\cdot x_{t})$$

K是卡尔曼增益kalman gain，不仅转换量纲，而且还能表达权重，这是核心

方差最小可求卡尔曼增益

$$K_{t}=P_{t}H^{T}(HP_{t}H^{T} + R) ^{-1}$$

其实标量情况下，
$$K=\frac{P}{P+R}$$

k=1，说明更相信观测值，k=0，更相信推测值

$$P_{t}=(1-K_{t}H)P_{t}$$

方差是变大的

其实卡尔曼滤波就是：

准确地推断出下一个时刻小车的位置

但是我们既不能相信我们的推断，也不能相信我们的观测

要基于推测和观测，之间，求一个加权值



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 模拟数据

t = np.linspace(1,100,100)

position_gt = 0.5 *t **2
position_observed = position_gt +np.random.normal(0,500,size=t.shape[0])
plt.plot(t,position_gt,label='truth we never know')
plt.plot(t,position_observed,label='observed position')
plt.legend()
plt.show()

In [ ]:
# 用上一个状态推测下一个

# 第一次估计就用第一个观测值
Q=50
predicts_kalman = [position_observed[0]]

position_predict = position_observed[0]
var_observed = 1000
var_predict = 100
for i in range(1,t.shape[0]):
    dv = position_gt[i] - position_gt[i-1] + np.random.normal(0,50)
    position_predict += dv
    var_predict += Q

    # kalman filter
    position_predict = position_predict* var_observed/(var_observed+var_predict) +position_observed[i] *var_predict / (var_observed+var_predict)

    var_predict = var_predict * var_observed/(var_predict+var_observed)**2
    predicts_kalman.append(position_predict)

plt.scatter(t,predicts_kalman, label='kalman')

plt.plot(t,position_gt,label='truth we never know')
plt.plot(t,position_observed,label='observed position')
plt.legend()
plt.show()

给定一个状态，如何变到下一个状态

包括状态，状态的协方差矩阵

注意这里是推断的协方差矩阵，而不是真实的，真实的永远只有一个，是未知的

## 用opencv实现track

In [ ]:
import cv2
import os
import random

In [ ]:
root = os.curdir

In [ ]:
def create_tracker_by_name(tracker_name):
    tracker_list = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    if tracker_name in tracker_list:
        return eval(f'cv2.Tracker{tracker_name}_create()')
    else:
        print(f'ERROR: tracker "{tracker_name}" not found in {tracker_list}')
        return None


In [ ]:
video_path = os.path.join(root,'short.mp4')
cap = cv2.VideoCapture(video_path)
status, frame = cap.read()
if status!= True:
    print('ERROR: failed to read video')

In [ ]:
bbox_list = []
color_list = []
while True:
    bbox=cv2.selectROI('MultiTracker', frame)
    print(type(bbox))
    bbox_list.append(bbox)
    color_list.append((random.randint(0,255),random.randint(0,255),random.randint(0,255)))
    print(bbox_list,color_list)
    print("Press q to quit selecting boxes and start tracking")
    print("Press any other key to select next object")
    if cv2.waitKey(0) & 0xFF ==ord('q'):
        break

In [ ]:
tracker_str = 'KCF'

In [ ]:
multi_tracker = cv2.MultiTracker_create()

In [ ]:
for bbox in bbox_list:
    multi_tracker.add(create_tracker_by_name(tracker_str),frame,bbox)

In [ ]:
while cap.isOpened():
    status, frame = cap.read()
    
    status, new_box_list = multi_tracker.update(frame)

    for index, new_box in enumerate(new_box_list):
        p1 = (int(new_box[0]), int(new_box[1]))
        p2 = (int(new_box[0] + new_box[2]),int(new_box[1]+new_box[3]))
        cv2.rectangle(frame,p1,p2,[255,0,255],1,1)
    cv2.imshow('MultiTracker', frame)

    if cv2.waitKey(2) & 0xFF ==27:#ESC
        break